In [101]:
import cv2
import numpy as np
import keyboard

s1_angle = np.pi/2 #sensor1_angle
s2_angle = 0
s3_angle = np.pi
s4_angle = -np.pi/2
    
dist = 10 # distance of sensor from end effector

# DANGER, lowering this might be bad
maximumSensorValue = 500 # this can be between 0-1024, just hard capping it
coeff = 0.3 # to downscale the applied force (original range is 0-1024
                
    
min_sensor_to_bother = 10 # sensor range is 0-1024 btw # minimum sensor value to bother
distance_from_desired = 0 # this must be set on a cooldown

def getSensorInfluence(actual, theta1, theta2):
    
    sensor1 = (int(dist * np.sin(theta1+theta2+s1_angle)), int(dist * np.cos(theta1+theta2+s1_angle)) )
    sensor2 = (int(dist * np.sin(theta1+theta2+s2_angle)), int(dist * np.cos(theta1+theta2+s2_angle)) )
    sensor3 = (int(dist * np.sin(theta1+theta2+s3_angle)), int(dist * np.cos(theta1+theta2+s3_angle)) )
    sensor4 = (int(dist * np.sin(theta1+theta2+s4_angle)), int(dist * np.cos(theta1+theta2+s4_angle)) )
    
    return sensor1, sensor2, sensor3, sensor4

def getSensorPositions(actual, sensor1, sensor2, sensor3, sensor4):
    
    sensor1_pos = (int(actual[0] + sensor1[0]), int(actual[1] + sensor1[1]) )
    sensor2_pos = (int(actual[0] + sensor2[0]), int(actual[1] + sensor2[1]) )
    sensor3_pos = (int(actual[0] + sensor3[0]), int(actual[1] + sensor3[1]) )
    sensor4_pos = (int(actual[0] + sensor4[0]), int(actual[1] + sensor4[1]) )
    
    return sensor1_pos, sensor2_pos, sensor3_pos, sensor4_pos

def forwardKin(theta1, theta2):
    x = L2*np.sin(theta1 + theta2) + L1*np.sin(theta1)
    y = L2*np.cos(theta1 + theta2) + L1*np.cos(theta1)
    return list((int(x) + center[0], int(y) + center[1]))

def forwardElbowKin(theta1):
    x = L1*np.sin(theta1)
    y = L1*np.cos(theta1)
    return list((int(x) + center[0], int(y) + center[1]))

# physics info
L1 = 70
L2 = 70

# window size 
width = 750
height = 750

center = (int(width/2), int(height/2))
theta1 = np.pi*1/4
theta2 = np.pi*1/4

end_effector = forwardKin(theta1, theta2)
elbow = forwardElbowKin(theta1)
desired = end_effector.copy()
actual = end_effector.copy()
sensor1, sensor2, sensor3, sensor4 = getSensorInfluence(actual, theta1, theta2) 
sensor1_pos, sensor2_pos, sensor3_pos, sensor4_pos = getSensorPositions(actual, sensor1, sensor2, sensor3, sensor4)

link1_image = cv2.imread("realArm/Photoshop/link1_clear.png", cv2.IMREAD_UNCHANGED)
link2_image = cv2.imread("realArm/Photoshop/link2_clear.png", cv2.IMREAD_UNCHANGED)
base_image = cv2.imread("realArm/Photoshop/base_clear.png")

scale_percent = 90 # percent of original size
w = int(base_image.shape[1] * scale_percent / 100)
h = int(base_image.shape[0] * scale_percent / 100)
dim = (w, h)
# resize image
resized = cv2.resize(base_image, dim, interpolation = cv2.INTER_AREA)

scale_percent2 = 30 # percent of original size
w2 = int(link1_image.shape[1] * scale_percent2 / 100)
h2 = int(link1_image.shape[0] * scale_percent2 / 100)
dim2 = (w2, h2)
# resize image
resized2 = cv2.resize(link1_image, dim2, interpolation = cv2.INTER_AREA)



font                   = cv2.FONT_HERSHEY_SIMPLEX
fontScale              = 0.55
fontColor              = (255,255,255)
thickness              = 1
lineType               = 2

import imutils
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

angle = 0
def draw():
    global angle
    window = np.ones((height, width, 3), dtype=np.uint8) *255
    cv2.putText(window,"Peak Velocity: " + str(26) + " m/s", (10,20), font, fontScale, fontColor, thickness, lineType)
    cv2.putText(window, "Velocity: " + str(20) + " m/s", (10,40), font, fontScale, fontColor, thickness, lineType)
    cv2.putText(window, "Fd: " + str(50) + " N", (10,60), font, fontScale, fontColor, thickness, lineType)
    cv2.putText(window, "Samples: " + str(6), (10,80), font, fontScale, fontColor, thickness, lineType)

    window[center[1]-round(resized.shape[0]/2):center[1]+(resized.shape[0]-round(resized.shape[0]/2)), center[0]-round(resized.shape[1]/2):center[0]+(resized.shape[1]-round(resized.shape[1]/2))] = resized

    #resized22 = imutils.rotate_bound(resized2, angle)
    angle += 1
    resized22 = rotate_image(resized2, angle)
    # normalize alpha channels from 0-255 to 0-1
    x1 = center[1]-round(resized2.shape[0]/2)
    y1 = center[0]-round(resized2.shape[1]/2)
    
    #window[x1:x1+resized22.shape[0], y1:y1+resized22.shape[1], :] = resized22
    # set adjusted colors
    #for i in range(0, resized22.shape[0]):
    #    for j in range(0, resized22.shape[1]):
    #        if resized22[i][j][3] > 125:
    #            window[x1+i, y1+j, :] = resized22[i, j, :]

    window = cvzone.overlayPNG(window, resized22, [x1, y1])
    
    window = cv2.line(window, tuple(center), tuple(elbow), (255, 0, 0), 6)
    window = cv2.line(window, tuple(elbow), tuple(end_effector), (255, 0, 0), 6)
    #window = cv2.line(window, tuple(center), tuple(desired), (255, 255, 255), 6)
    
    #window = cv2.line(window, tuple(center), tuple(ogelbow), (0, 255, 0), 6)
    #window = cv2.line(window, tuple(ogelbow), tuple(ogend_effector), (0, 255, 0), 6)
    
    # draw force sensors
    window = cv2.circle(window, sensor1_pos, 2, (0, 255, 0), 2)
    window = cv2.circle(window, sensor2_pos, 2, (226, 253, 48), 2)
    window = cv2.circle(window, sensor3_pos, 2, (48, 82, 253), 2)
    window = cv2.circle(window, sensor4_pos, 2, (253, 48, 239), 2)
    
    window = cv2.circle(window, tuple(desired), 5, (255, 255, 255), 2)   # we have casual coordinates but we want zero to be the middle of screen with a range of -4 to 4 just bcz it works with our other values
    return window


# start main Gui thread
cv2.namedWindow('output', cv2.WINDOW_AUTOSIZE)
while True:
    window = draw()
    cv2.imshow('output', window)

    # for quitting
    cv2.waitKey(1)
    if keyboard.is_pressed("esc"):
        break

# clean up
cv2.destroyAllWindows()